<a href="https://colab.research.google.com/github/abebual/LLMs-for-Flight-Safety/blob/main/download_faa_resource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Source

https://drs.faa.gov/browse

Dynamic Regulatory System - A Comprehensive Knowledge Center of Regulatory and Guidance Material from the Office of Aviation Safety and other Services and Offices


In [12]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import os

faa_directive_links = {"2013 Code of Federal Regulations Title 14 Part 33.pdf" : 'https://www.govinfo.gov/content/pkg/CFR-2013-title14-vol1/pdf/CFR-2013-title14-vol1-part33.pdf',
    "FAR 25_119 Landing Climb - All-Engines-Operating.pdf" : 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-part25-subpartB-subjectgroup-id605.pdf',
    "FAR 25_903 Engines.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-part25-subpartE-subjectgroup-id832.pdf',
    "FAR 25_1093 Induction System Icing Protection.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-part25-subpartE-subjectgroup-id832.pdf',
    "FAR 25 Appendix C.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec25-1701.pdf',
    "FAR 25 Appendix O - Supercooled Large Drop Icing Conditions.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2016-title14-vol1/pdf/CFR-2016-title14-vol1-part25-appO.pdf',
    "FAR 33_5 Instruction Manual for Installing and Operating the Engine.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-part33.pdf',
    "FAR 33_7 Engine Ratings and Operating Limitations.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-7.pdf',
    "FAR 33_27 Turbine, Compressor - Fan and Turbosupercharger Rotors.pdf" : 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-27.pdf',
    "FAR 33_65 Surge And Stall Characteristics.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-65.pdf',
    "FAR 33_68 Induction System Icing.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-68.pdf',
    "FAR 33_69 Ignitions System.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-69.pdf',
    "FAR 33_73 Power Or Thrust Response.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-73.pdf',
    "FAR 33_77 Foreign Object Ingestion-Ice.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-77.pdf',
    "FAR 33_78 Rain and Hail Ingestion.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-78.pdf',
    "FAR 33_87 Endurance Test (Non-Rotorcraft Sections).pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-87.pdf',
    "FAR 33_89 Operation Test.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-89.pdf',
    "FAR 33_201 Design and Test Requirements for Early ETOPS Eligibility.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2022-title14-vol1/pdf/CFR-2022-title14-vol1-sec33-201.pdf',
    "FAR 21_3 Reporting of failures malfunctions and defects.pdf": 'https://www.govinfo.gov/content/pkg/CFR-2023-title14-vol1/pdf/CFR-2023-title14-vol1-sec21-3.pdf',
    "AC 33.4-1.pdf" : 'https://drs.faa.gov/browse/excelExternalWindow/91546F9ED22422A9862569C4006778F3.0001?modalOpened=true',
    "AC 33.4-2.pdf" : 'https://drs.faa.gov/browse/excelExternalWindow/6CF96E66FDE2BD0E86256A84005B1F8A.0001?modalOpened=true',
    "AC 33.4-3.pdf" : 'https://drs.faa.gov/browse/excelExternalWindow/4173A686D103940186257083006B60B6.0001?modalOpened=true',
    "AC 33.7-1.pdf" : 'https://drs.faa.gov/browse/excelExternalWindow/BED276728541C3908625775100629581.0001?modalOpened=true',
    "AC_33.14-1_Chg 1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/DRSDOCID101331202820230713155936.0001?modalOpened=true',
    "AC 33.15-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/618F8B0208E42DDE862569D10077F4A8.0001?modalOpened=true',
    "AC 33.15-2.pdf" : 'https://drs.faa.gov/browse/excelExternalWindow/558068BDAC33806F86257830005D15BD.0001?modalOpened=true',
    "AC 33.15-3_Final.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/DRSDOCID192416064820230418232801.0001?modalOpened=true',
    "AC 33.17-1A.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/31B5E240C190152586257C2800664B14.0001?modalOpened=true',
    "AC 33.19-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/59F49483B9998AA186256F25006D98C5.0001?modalOpened=true',
    "AC 33.27-1A.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/8357F4AF18623238862578D200544B86.0001?modalOpened=true',
    "AC 33.28-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/A4233080C2FA917986256BF0006EB819.0001?modalOpened=true',
    "AC 33.28-2.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/73E5F5AAE088D0C686256D8D0071FF1C.0001?modalOpened=true',
    "AC 33.28-3.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/454BD18FC8D316B786257CE80070A362.0001?modalOpened=true',
    "AC 33.47-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/33773A867924FD65862569B3005D6B32.0001?modalOpened=true',
    "AC 33.63-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/497CB6EA06AAB1B88625732F005A2426.0001?modalOpened=true',
    "AC 33.64-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/83E0890DCCD8E318862577A4005AC0F0.0001?modalOpened=true',
    "AC_33.70-1_CHG1_Incorporated.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/46F4E2512561476086257609006A293B.0001?modalOpened=true',
    "AC 33.70-2.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/D7C51BEF8CCD42E08625762500475648.0001?modalOpened=true',
    "AC_33.70-3.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/DRSDOCID148835457420230301021813.0001?modalOpened=true',
    "AC 33.70-4_Final.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/DRSDOCID100150982820230710205923.0001?modalOpened=true',
    "33.74-92-1B.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/0A897725B48E943786257FE300556050.0001?modalOpened=true',
    "33.75-1A-chg1_incorporated.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/365A2E7285B2F2BB86257369006D8AF4.0001?modalOpened=true',
    "AC 33.76-1B.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/DRSDOCID144438676820230329172202.0001?modalOpened=true',
    "AC 33.78-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/B6560911BB36F087862569B3005D6BF0.0001?modalOpened=true',
    "AC 33.83-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/D371322EAE8189AB8625763100713BA2.0001?modalOpened=true',
    "AC-33.83-2B.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/DRSDOCID125827870420230203164059.0001?modalOpened=true',
    "AC 33.87-1A.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/466031CB5C7C94D486257E04006458B7.0001?modalOpened=true',
    "AC 33.87-2.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/1122B06020B7EEFD862575EB005BAFD5.0001?modalOpened=true',
    "AC 33.90-1A.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/6CD4EC7D0F45123E86257F9C006F7BFA.0001?modalOpened=true',
    "AC 33.91-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/84F46B4FADB3C045862577F800676F17.0001?modalOpened=true',
    "AC 33.201-1.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/D8B0E1CB7BCA2D44862576B9004A287D.0001?modalOpened=true',
    "AC 33-2C.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/A99885454536641486257B98006BED3E.0001?modalOpened=true',
    "AC 33-5.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/EAB0330A3C7A22EB862569B3005D6ACE.0001?modalOpened=true',
    "AC_33-6_CHG1_Incorporated.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/996E9C7DE033074D862569B3005D6B00.0001?modalOpened=true',
    "AC 33-7A.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/86A12FBF37E90C2E862575D700497554.0001?modalOpened=true',
    "AC 33-8.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/149E4B567FD69DA1862576170069C14D.0001?modalOpened=true',
    "AC 33-9.pdf": 'https://drs.faa.gov/browse/excelExternalWindow/31916B17FF7305D48625771C006F2ECA.0001?modalOpened=true',
    "AC 33-11.pdf": 'https://drs.faa.gov/browse/AC/doctypeDetails?modalOpened=true'

}



download_directory = "FAA Directives"
os.makedirs(download_directory, exist_ok=True)


def download_directives(pdf_url, local_path):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(local_path, 'wb') as pdf_file:
            pdf_file.write(response.content)
        print(f"Downloaded: {local_path}")

    else:
        print(f"Failed to download: {pdf_url}")



for file_name, pdf_url in faa_directive_links.items():
    local_path = os.path.join(download_directory, file_name)
    download_directives(pdf_url, local_path)




Downloaded: FAA Directives/2013 Code of Federal Regulations Title 14 Part 33.pdf
Downloaded: FAA Directives/FAR 25_119 Landing Climb - All-Engines-Operating.pdf
Downloaded: FAA Directives/FAR 25_903 Engines.pdf
Downloaded: FAA Directives/FAR 25_1093 Induction System Icing Protection.pdf
Downloaded: FAA Directives/FAR 25 Appendix C.pdf
Downloaded: FAA Directives/FAR 25 Appendix O - Supercooled Large Drop Icing Conditions.pdf
Downloaded: FAA Directives/FAR 33_5 Instruction Manual for Installing and Operating the Engine.pdf
Downloaded: FAA Directives/FAR 33_7 Engine Ratings and Operating Limitations.pdf
Downloaded: FAA Directives/FAR 33_27 Turbine, Compressor - Fan and Turbosupercharger Rotors.pdf
Downloaded: FAA Directives/FAR 33_65 Surge And Stall Characteristics.pdf
Downloaded: FAA Directives/FAR 33_68 Induction System Icing.pdf
Downloaded: FAA Directives/FAR 33_69 Ignitions System.pdf
Downloaded: FAA Directives/FAR 33_73 Power Or Thrust Response.pdf
Downloaded: FAA Directives/FAR 33_7